In [1]:
from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image
from typing import List, Union
import logging
import time
import io
from concurrent.futures import ThreadPoolExecutor

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ImageCaptioningPipeline:
    def __init__(self, model_path: str = "nlpconnect/vit-gpt2-image-captioning",
                 max_length: int = 16, num_beams: int = 4, batch_size: int = 8):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.max_length = max_length
        self.num_beams = num_beams
        self.batch_size = batch_size
        
        logger.info(f"Loading model from {model_path}")
        self.model = VisionEncoderDecoderModel.from_pretrained(model_path)
        self.feature_extractor = ViTImageProcessor.from_pretrained(model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        
        self.model.to(self.device)
        if self.device.type == 'cuda':
            self.model = self.model.half()  # FP16 for faster inference
        self.model.eval()
        
        self.model.gradient_checkpointing_enable()

        # Warmup
        logger.info("Warming up model...")
        dummy_image = torch.randn(1, 3, 224, 224, device=self.device)
        if self.device.type == 'cuda':
            dummy_image = dummy_image.half()
        with torch.no_grad():
            _ = self.model.generate(dummy_image, 
                                   max_length=self.max_length,
                                   num_beams=self.num_beams)
        
        self.gen_kwargs = {
            "max_length": max_length, 
            "num_beams": num_beams,
            "pad_token_id": self.tokenizer.pad_token_id,
            "use_cache": True  
        }

    @staticmethod
    def load_image(image_path: Union[str, bytes, Image.Image]) -> Image.Image:
        if isinstance(image_path, str):
            image = Image.open(image_path)
        elif isinstance(image_path, bytes):
            image = Image.open(io.BytesIO(image_path))
        elif isinstance(image_path, Image.Image):
            image = image_path
        else:
            raise ValueError(f"Unsupported image type: {type(image_path)}")
        
        if image.mode != "RGB":
            image = image.convert(mode="RGB")
        return image

    def preprocess_images(self, image_paths: List[Union[str, bytes, Image.Image]]) -> torch.Tensor:
        with ThreadPoolExecutor() as executor:
            images = list(executor.map(self.load_image, image_paths))
        pixel_values = self.feature_extractor(images=images, return_tensors="pt").pixel_values
        pixel_values = pixel_values.pin_memory() if self.device.type == 'cuda' else pixel_values
        if self.device.type == 'cuda':
            pass
            # pixel_values = pixel_values.half()
        return pixel_values.to(self.device, non_blocking=True)

    @torch.no_grad()
    def __call__(self, image_paths: List[Union[str, bytes, Image.Image]]) -> List[str]:
        pixel_values = self.preprocess_images(image_paths)
        
        with torch.amp.autocast('cuda', enabled=self.device.type == 'cuda'):  # Mixed precision
            output_ids = self.model.generate(
                pixel_values,
                **self.gen_kwargs
            )
        
        preds = self.tokenizer.batch_decode(output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        return [pred.strip() for pred in preds]


/home/automl/Xuanan/HMI/VisionMate/AI/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-12 21:57:24.782465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-12 21:57:24.818759: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-12 21:57:24.829835: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-12 21:57:24.857031: I tensorflow/core/platform/cpu_feature_guard.cc:210] This Te

In [2]:
pipeline = ImageCaptioningPipeline()


INFO:__main__:Loading model from nlpconnect/vit-gpt2-image-captioning
VisionEncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
INFO:__main__:Warming up model...
The attention mask is not set and cannot be inferr

In [3]:
image_paths = ['/home/automl/Xuanan/HMI/VisionMate/AI/image.png']
    
logger.info("Running PyTorch pipeline...")
start_time = time.time()

captions = pipeline(image_paths)
    
end_time = time.time()
logger.info(f"Inference time: {end_time - start_time:.2f} seconds")
    
for path, caption in zip(image_paths, captions):
    logger.info(f"{path}: {caption}")


INFO:__main__:Running PyTorch pipeline...
INFO:__main__:Inference time: 0.61 seconds
INFO:__main__:/home/automl/Xuanan/HMI/VisionMate/AI/image.png: a soccer game in progress with a player jumping for the ball
